In [83]:
import pandas as pd
import parselmouth

from parselmouth.praat import call
import parselmouth
from fnmatch import fnmatch
from tqdm.notebook import tqdm

import os

In [84]:


def getFeatures(voiceID, f0min, f0max, unit):
    try:
        sound = parselmouth.Sound(voiceID) # read the sound
        duration = call(sound, "Get total duration") # duration
        pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object


        meanF0 = call(pitch, "Get mean", 0, 0, unit) #MDVP:Fo(Hz)
        # get standard deviation
        stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) 
        harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
        hnr = call(harmonicity, "Get mean", 0, 0) #HNR
        pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
        localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:Jitter(%)
        localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:Jitter(Abs)
        rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:RAP
        ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:PPQ
        ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3) #Jitter:DDP
        localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #MDVP:Shimmer
        localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #MDVP:Shimmer(dB)
        apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #Shimmer:APQ3
        aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #Shimmer:APQ5
        apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

        
        #print("file " + voiceID + " was read successfuly")
        return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer
    except Exception as exc:
        #print(exc)
        print("file " + voiceID + " was not read because it is not a sound file - ")
        return -1

In [85]:
"""
def getListOfAudioFiles(dir_path):
    #print list of sound files in directory
    res = []
    # Iterate directory
    for file_path in os.listdir(dir_path):
        # check if current file_path is a file
        if os.path.isfile(os.path.join(dir_path, file_path)):
            # add filename to list
            res.append(file_path)
    return(res)
    """

def getListOfAudioFiles(dir_path):
    res= []
    pattern = "*.wav"

    for path, subdirs, files in os.walk(dir_path):
        for name in files:
            if fnmatch(name, pattern):
                res.append(os.path.join(name))
    return(res)




In [86]:
italian_pd_audiofiles_path = '../data/Italian_PD'
italian_control_audiofiles_path = '../data/Italian_HC'
mdvr_pd_audiofiles_path = '../data/MDVR_PD'
mdvr_control_audiofiles_path = '../data/MDVR_HC'

def getFeaturesFromAudioFiles(pd_audiofiles_path, control_audiofiles_path):
    pd_audiofiles_names = getListOfAudioFiles(pd_audiofiles_path)
    control_audiofiles_names = getListOfAudioFiles(control_audiofiles_path)


    original_features_df = pd.DataFrame(columns=['Duration', 'meanF0', 'stdevF0', 'hnr', 'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'aqpq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'PD'])


    for i in tqdm(range(len(pd_audiofiles_names)-1)):
        values = getFeatures(pd_audiofiles_path+'/'+pd_audiofiles_names[i], 75, 500, "Hertz")
        if values!=-1:
            duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = values
            original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer,1]

    for i in tqdm(range(len(control_audiofiles_names)-1)):
        values = getFeatures(control_audiofiles_path+'/'+control_audiofiles_names[i], 75, 500, "Hertz")
        if values!=-1:
            duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = values
            original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer,0]

    return original_features_df


In [87]:
mdvr_original_featuers_df = getFeaturesFromAudioFiles(mdvr_pd_audiofiles_path, mdvr_control_audiofiles_path)

  0%|          | 0/341 [00:00<?, ?it/s]

file ../data/MDVR_PD/chunk12 6.wav was read successfuly
file ../data/MDVR_PD/chunk1 16.wav was read successfuly
file ../data/MDVR_PD/chunk8 7.wav was read successfuly
file ../data/MDVR_PD/chunk10 4.wav was read successfuly
file ../data/MDVR_PD/chunk6 8.wav was read successfuly
file ../data/MDVR_PD/chunk2.wav was read successfuly
file ../data/MDVR_PD/chunk16 2.wav was read successfuly
file ../data/MDVR_PD/chunk32 3.wav was read successfuly
file ../data/MDVR_PD/chunk21 9.wav was read successfuly
file ../data/MDVR_PD/chunk3 13.wav was read successfuly
file ../data/MDVR_PD/chunk3 12.wav was read successfuly
file ../data/MDVR_PD/chunk21 8.wav was read successfuly
file ../data/MDVR_PD/chunk32 2.wav was read successfuly
file ../data/MDVR_PD/chunk16 3.wav was read successfuly
file ../data/MDVR_PD/chunk3.wav was read successfuly
file ../data/MDVR_PD/chunk6 9.wav was read successfuly
file ../data/MDVR_PD/chunk10 5.wav was read successfuly
file ../data/MDVR_PD/chunk8 6.wav was read successfuly
fi

  0%|          | 0/473 [00:00<?, ?it/s]

file ../data/MDVR_HC/chunk12 6.wav was read successfuly
file ../data/MDVR_HC/chunk1 16.wav was read successfuly
file ../data/MDVR_HC/chunk8 7.wav was read successfuly
file ../data/MDVR_HC/chunk10 4.wav was read successfuly
file ../data/MDVR_HC/chunk6 8.wav was read successfuly
file ../data/MDVR_HC/chunk17 12.wav was read successfuly
file ../data/MDVR_HC/chunk2.wav was read successfuly
file ../data/MDVR_HC/chunk16 2.wav was read successfuly
file ../data/MDVR_HC/chunk32 3.wav was read successfuly
file ../data/MDVR_HC/chunk21 9.wav was read successfuly
file ../data/MDVR_HC/chunk3 13.wav was read successfuly
file ../data/MDVR_HC/chunk3 12.wav was read successfuly
file ../data/MDVR_HC/chunk21 8.wav was read successfuly
file ../data/MDVR_HC/chunk32 2.wav was read successfuly
file ../data/MDVR_HC/chunk16 3.wav was read successfuly
file ../data/MDVR_HC/chunk17 13.wav was read successfuly
file ../data/MDVR_HC/chunk3.wav was read successfuly
file ../data/MDVR_HC/chunk6 9.wav was read successfuly

In [88]:
italian_original_features_df = getFeaturesFromAudioFiles(italian_pd_audiofiles_path, italian_control_audiofiles_path)


  0%|          | 0/274 [00:00<?, ?it/s]

file ../data/Italian_PD/VA1VSIOTLOP47M100220171331.wav was read successfuly
file ../data/Italian_PD/VU1sncihcio44M1606161724.wav was read successfuly
file ../data/Italian_PD/VI2cdaopmoe67M2605161911.wav was read successfuly
file ../data/Italian_PD/VI1rlouscsi77F2605161827.wav was read successfuly
file ../data/Italian_PD/VE2ssacvhei61M1606161745.wav was read successfuly
file ../data/Italian_PD/VI2GLIAUDLO50F100220171303.wav was read successfuly
file ../data/Italian_PD/VU1VSIPTIOZ46M240120171932.wav was read successfuly
file ../data/Italian_PD/VE2VSIPTIOZ46M240120171927.wav was read successfuly
file ../data/Italian_PD/VE2GNIEOGVL47M100220171217.wav was read successfuly
file ../data/Italian_PD/VO1GPIUUGLL63F100220171026.wav was read successfuly
file ../data/Italian_PD/VI1sncihcio44M1606161722.wav was read successfuly
file ../data/Italian_PD/VI1AGNUTGOL52F100220171051.wav was read successfuly
file ../data/Italian_PD/VA2lloeroun56F2605161927.wav was read successfuly
file ../data/Italian_PD/

  0%|          | 0/219 [00:00<?, ?it/s]

file ../data/Italian_HC/VO2MPAERRIR56F300320171148.wav was read successfuly
file ../data/Italian_HC/VO2GCIALSDA52F170320171130.wav was read successfuly
file ../data/Italian_HC/VU2AGNIGNEE54F230320171023.wav was read successfuly
file ../data/Italian_HC/VU1GGIAORVG47F300320171215.wav was read successfuly
file ../data/Italian_HC/VA2GLIIUSSC57M210320171053.wav was read successfuly
file ../data/Italian_HC/VA1GGRIAUZL45F020420171821.wav was read successfuly
file ../data/Italian_HC/VU2VAILTFOO49M230320171034.wav was read successfuly
file ../data/Italian_HC/VI1GLIIUSSC57M210320171054.wav was read successfuly
file ../data/Italian_HC/VU2APNITNOT56F230320170855.wav was read successfuly
file ../data/Italian_HC/VO1VAILTFOO49M230320171033.wav was read successfuly
file ../data/Italian_HC/VE2GGIAORVG47F300320171214.wav was read successfuly
file ../data/Italian_HC/VI2GBIAORVI48M230320171238.wav was read successfuly
file ../data/Italian_HC/VE2ACNUTCOC40M230320171126.wav was read successfuly
file ../data

In [89]:
italian_outname = 'italian_original_features.csv'
mdvr_outname = 'mdvr_original_featuers.csv'

outdir = '../data/tables/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

italian_original_features_df.to_csv(os.path.join(outdir, italian_outname),index=False)
mdvr_original_featuers_df.to_csv(os.path.join(outdir, mdvr_outname),index=False)